# facial keypoints detector

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import path as opath
import os
import sys
import seaborn as sns
from PIL import Image
import errno
sns.set(style="whitegrid", color_codes=True)

# add utils into PYTHONPATH
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

from utils import get_batches, get_accuracy

Using TensorFlow backend.


In [2]:
# ENV settings
DATA_PATH = 'data/'
MODEL_PATH = 'models/'
%mkdir -p $MODEL_PATH

TRAIN_PATH = opath.join(DATA_PATH, 'train')
TEST_PATH = opath.join(DATA_PATH, 'test')
VALID_PATH = opath.join(DATA_PATH, 'valid')

## Downloading data

In [3]:
%mkdir -p $DATA_PATH
%pwd
%cd $DATA_PATH
%pwd
!kg download -c facial-keypoints-detector
%ls
%cd ..
%pwd

/home/ansible/workspace/kaggle_fun/facial_keypoints_detector/data
downloading https://www.kaggle.com/c/facial-keypoints-detector/download/train.csv

train.csv 100% |#####################################| Time: 0:00:01  20.0 MiB/s

downloading https://www.kaggle.com/c/facial-keypoints-detector/download/test.csv

test.csv 100% |######################################| Time: 0:00:00  13.6 MiB/s

downloading https://www.kaggle.com/c/facial-keypoints-detector/download/train_identity.csv

train_identity.csv 100% |############################| Time: 0:00:00  62.8 KiB/s

test.csv  train.csv  train_identity.csv
/home/ansible/workspace/kaggle_fun/facial_keypoints_detector


u'/home/ansible/workspace/kaggle_fun/facial_keypoints_detector'

## Preprocess data

In [4]:
def convert_pixels(pix_str):
    return np.array([int(p) for p in pix_str.split(' ')], 'uint8').reshape((48, 48))


def load_data():
    df = pd.read_csv(opath.join(DATA_PATH, 'train.csv'), converters={'Pixels': convert_pixels})
    return df

data = load_data()

In [5]:
def decode_label(df):
    df = df.copy()
    df.loc[df['Emotion'] == 0, 'Emotion'] = 'anger'
    df.loc[df['Emotion'] == 1, 'Emotion'] = 'disgust'
    df.loc[df['Emotion'] == 2, 'Emotion'] = 'fear'
    df.loc[df['Emotion'] == 3, 'Emotion'] = 'happy'
    df.loc[df['Emotion'] == 4, 'Emotion'] = 'sad'
    df.loc[df['Emotion'] == 5, 'Emotion'] = 'surprise'
    df.loc[df['Emotion'] == 6, 'Emotion'] = 'neutral'
    return df

In [6]:
decoded_data = decode_label(data)
remove_neutral_data = decoded_data[decoded_data['Emotion'] == 'neutral'].sample(700)
remove_happy_data = decoded_data[decoded_data['Emotion'] == 'happy'].sample(300)
transformed_data = decoded_data.drop((remove_happy_data+remove_neutral_data).index)
test_data = transformed_data.sample(frac=0.1)
transformed_data = transformed_data.drop(test_data.index)
valid_data = transformed_data.sample(frac=0.2)
train_data = transformed_data.drop(valid_data.index)

In [7]:
def mkdir(path):
    try:
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise
            
            
def save_img_files(df, base):
    sub_path = opath.join(DATA_PATH, base)
    for idx, row in df.iterrows():
        category_path = opath.join(sub_path, row['Emotion'])
        mkdir(category_path)
        Image.fromarray(row['Pixels']).save(opath.join(category_path, '{}.png'.format(idx)), 'png')

In [8]:
save_img_files(test_data, 'test')
save_img_files(valid_data, 'valid')
save_img_files(train_data, 'train')

## Vanilla model

In [9]:
def get_bench_model():
    model = Sequential()
    model.add(Conv2D(16, 3, activation='relu', padding='same', input_shape=(3, 48, 48)))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(32, 3, activation='relu', padding='same',))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, 3, activation='relu', padding='same',))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='softmax'))
    return model

bench_model = get_bench_model()
bench_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 48, 48)        448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 24, 24)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 24, 24)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 12, 12)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 12, 12)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 6, 6)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)              0         
__________

In [10]:
bench_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=90)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator()

In [12]:
batch_size = 32
bench_weights_path = opath.join(MODEL_PATH, 'bench.weights.best.hdf5')

checkpointer = ModelCheckpoint(filepath=bench_weights_path, verbose=1, save_best_only=True)

train_batches = get_batches(TRAIN_PATH, train_datagen, (48, 48))
valid_batches = get_batches(VALID_PATH, valid_datagen, (48, 48))

bench_model.fit_generator(train_batches, steps_per_epoch=train_batches.samples//batch_size, 
                          validation_data=valid_batches, validation_steps=valid_batches.samples//batch_size,
                          callbacks=[checkpointer],  epochs=30)


Found 2288 images belonging to 7 classes.
Found 572 images belonging to 7 classes.
Epoch 1/30
72/71 [==============================] - 4s 59ms/step - loss: 1.9490 - acc: 0.1444 - val_loss: 1.9376 - val_acc: 0.1591
Epoch 2/30
72/71 [==============================] - 2s 31ms/step - loss: 1.9286 - acc: 0.1984 - val_loss: 1.8920 - val_acc: 0.2797
Epoch 3/30
72/71 [==============================] - 2s 34ms/step - loss: 1.8713 - acc: 0.2571 - val_loss: 1.8115 - val_acc: 0.2762
Epoch 4/30
72/71 [==============================] - 2s 28ms/step - loss: 1.8077 - acc: 0.2796 - val_loss: 1.7334 - val_acc: 0.3287
Epoch 5/30
72/71 [==============================] - 2s 30ms/step - loss: 1.7381 - acc: 0.3248 - val_loss: 1.7186 - val_acc: 0.3409
Epoch 6/30
72/71 [==============================] - 2s 29ms/step - loss: 1.7017 - acc: 0.3337 - val_loss: 1.6430 - val_acc: 0.3706
Epoch 7/30
72/71 [==============================] - 2s 31ms/step - loss: 1.6262 - acc: 0.3792 - val_loss: 1.6336 - val_acc: 0.3864


Epoch 29/30
72/71 [==============================] - 2s 34ms/step - loss: 1.2030 - acc: 0.5520 - val_loss: 1.0797 - val_acc: 0.5839
Epoch 30/30
72/71 [==============================] - 2s 33ms/step - loss: 1.2282 - acc: 0.5477 - val_loss: 1.0767 - val_acc: 0.5804


In [13]:
bench_model.load_weights(bench_weights_path)
test_batches = get_batches(TEST_PATH, test_datagen, (48, 48))

print "benchmark model get accuary {}%".format(get_accuracy(bench_model, test_batches))

Found 318 images belonging to 7 classes.
benchmark model get accuary 0.487421383835%


## VGG16 finetune

### top layer

In [14]:
from utils.vgg16 import get_model
vgg_model = get_model()

vgg_model.pop()
for layer in vgg_model.layers:
    layer.trainable = False

vgg_model.add(Dense(7, activation='softmax'))

vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

In [15]:
vgg_model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
train_datagen = ImageDataGenerator()
valid_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

In [17]:
top_weights_path = opath.join(MODEL_PATH, 'vgg_tune_top.weights.best.hdf5')
checkpointer = ModelCheckpoint(filepath=top_weights_path, verbose=1, save_best_only=True)

train_batches = get_batches(TRAIN_PATH, train_datagen)
valid_batches = get_batches(VALID_PATH, valid_datagen)

vgg_model.fit_generator(train_batches, steps_per_epoch=train_batches.samples//train_batches.batch_size,
                          validation_data=valid_batches, validation_steps=valid_batches.samples//valid_batches.batch_size,
                          callbacks=[checkpointer],  epochs=20, verbose=1)


Found 2288 images belonging to 7 classes.
Found 572 images belonging to 7 classes.
Epoch 1/20
72/71 [==============================] - 12s 163ms/step - loss: 2.7104 - acc: 0.2411 - val_loss: 1.8916 - val_acc: 0.3619
Epoch 2/20
72/71 [==============================] - 10s 135ms/step - loss: 2.1936 - acc: 0.3408 - val_loss: 1.4307 - val_acc: 0.4423
Epoch 3/20
72/71 [==============================] - 10s 136ms/step - loss: 1.9723 - acc: 0.3869 - val_loss: 1.4091 - val_acc: 0.5105
Epoch 4/20
72/71 [==============================] - 10s 136ms/step - loss: 1.8889 - acc: 0.4202 - val_loss: 1.2878 - val_acc: 0.5227
Epoch 5/20
72/71 [==============================] - 9s 128ms/step - loss: 1.8453 - acc: 0.4371 - val_loss: 1.3000 - val_acc: 0.5052
Epoch 6/20
72/71 [==============================] - 9s 128ms/step - loss: 1.7558 - acc: 0.4396 - val_loss: 1.2943 - val_acc: 0.5035
Epoch 7/20
72/71 [==============================] - 9s 128ms/step - loss: 1.6872 - acc: 0.4768 - val_loss: 1.4373 - val_a

In [28]:
vgg_model.load_weights(top_weights_path)
test_batches = get_batches(TEST_PATH, test_datagen, (224, 224))

print "vgg16 model finetune top layer get accuary {}%".format(get_accuracy(vgg_model, test_batches))

Found 318 images belonging to 7 classes.
vgg16 model finetune top layer get accuary 0.619496855908%


### all dense layer

In [19]:
for idx, layer in enumerate(vgg_model.layers):
    print idx, layer.name

0 lambda_4
1 zero_padding2d_40
2 conv2d_40
3 zero_padding2d_41
4 conv2d_41
5 max_pooling2d_16
6 zero_padding2d_42
7 conv2d_42
8 zero_padding2d_43
9 conv2d_43
10 max_pooling2d_17
11 zero_padding2d_44
12 conv2d_44
13 zero_padding2d_45
14 conv2d_45
15 zero_padding2d_46
16 conv2d_46
17 max_pooling2d_18
18 zero_padding2d_47
19 conv2d_47
20 zero_padding2d_48
21 conv2d_48
22 zero_padding2d_49
23 conv2d_49
24 max_pooling2d_19
25 zero_padding2d_50
26 conv2d_50
27 zero_padding2d_51
28 conv2d_51
29 zero_padding2d_52
30 conv2d_52
31 max_pooling2d_20
32 flatten_4
33 dense_12
34 dropout_7
35 dense_13
36 dropout_8
37 dense_15


In [29]:
for layer in vgg_model.layers[:33]:
    layer.trainable = False
for layer in vgg_model.layers[33:]:
    layer.trainable = True

In [31]:
from keras.optimizers import Adam, SGD
optmizer = SGD(0.001, 0.9, 0.0001, True)
vgg_model.compile(optimizer=optmizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
dense_weights_path = opath.join(MODEL_PATH, 'vgg_tune_dense.weights.best.hdf5')
checkpointer = ModelCheckpoint(filepath=dense_weights_path, verbose=1, save_best_only=True)

train_batches = get_batches(TRAIN_PATH, train_datagen)
valid_batches = get_batches(VALID_PATH, valid_datagen)

vgg_model.fit_generator(train_batches, steps_per_epoch=train_batches.samples//train_batches.batch_size,
                          validation_data=valid_batches, validation_steps=valid_batches.samples//valid_batches.batch_size,
                          callbacks=[checkpointer],  epochs=5, verbose=1)


Found 2288 images belonging to 7 classes.
Found 572 images belonging to 7 classes.
Epoch 1/5
72/71 [==============================] - 95s 1s/step - loss: 1.6240 - acc: 0.4578 - val_loss: 0.9521 - val_acc: 0.6224
Epoch 2/5
72/71 [==============================] - 56s 776ms/step - loss: 1.0083 - acc: 0.6486 - val_loss: 0.8867 - val_acc: 0.6748
Epoch 3/5
72/71 [==============================] - 44s 614ms/step - loss: 0.7933 - acc: 0.7078 - val_loss: 0.7222 - val_acc: 0.7343
Epoch 4/5
72/71 [==============================] - 44s 612ms/step - loss: 0.6687 - acc: 0.7542 - val_loss: 0.7496 - val_acc: 0.7238
Epoch 5/5
72/71 [==============================] - 36s 500ms/step - loss: 0.6016 - acc: 0.7821 - val_loss: 0.7086 - val_acc: 0.7395


In [33]:
vgg_model.load_weights(dense_weights_path)
test_batches = get_batches(TEST_PATH, test_datagen, (224, 224))

print "vgg16 model finetune dense layer get accuary {}%".format(get_accuracy(vgg_model, test_batches))

Found 318 images belonging to 7 classes.
vgg16 model finetune dense layer get accuary 0.751572325545%


### tune conv

In [35]:
for layer in vgg_model.layers[:25]:
    layer.trainable = False
for layer in vgg_model.layers[25:]:
    layer.trainable = True

optmizer = SGD(0.0001, 0.9, 0.00001, True)
vgg_model.compile(optimizer=optmizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [36]:
conv_weights_path = opath.join(MODEL_PATH, 'vgg_tune_conv.weights.best.hdf5')
checkpointer = ModelCheckpoint(filepath=conv_weights_path, verbose=1, save_best_only=True)

train_batches = get_batches(TRAIN_PATH, train_datagen)
valid_batches = get_batches(VALID_PATH, valid_datagen)

vgg_model.fit_generator(train_batches, steps_per_epoch=train_batches.samples//train_batches.batch_size,
                          validation_data=valid_batches, validation_steps=valid_batches.samples//valid_batches.batch_size,
                          callbacks=[checkpointer],  epochs=5, verbose=1)

Found 2288 images belonging to 7 classes.
Found 572 images belonging to 7 classes.
Epoch 1/5
72/71 [==============================] - 197s 3s/step - loss: 0.3381 - acc: 0.8754 - val_loss: 0.6299 - val_acc: 0.7850
Epoch 2/5
72/71 [==============================] - 23s 324ms/step - loss: 0.2154 - acc: 0.9210 - val_loss: 0.6326 - val_acc: 0.7972
Epoch 3/5
72/71 [==============================] - 20s 271ms/step - loss: 0.1340 - acc: 0.9514 - val_loss: 0.6111 - val_acc: 0.7867
Epoch 4/5
72/71 [==============================] - 36s 496ms/step - loss: 0.1001 - acc: 0.9708 - val_loss: 0.6123 - val_acc: 0.8024
Epoch 5/5
72/71 [==============================] - 38s 522ms/step - loss: 0.0825 - acc: 0.9759 - val_loss: 0.6285 - val_acc: 0.8042


In [37]:
vgg_model.load_weights(conv_weights_path)
test_batches = get_batches(TEST_PATH, test_datagen, (224, 224))

print "vgg16 model finetune conv layer get accuary {}%".format(get_accuracy(vgg_model, test_batches))

Found 318 images belonging to 7 classes.
vgg16 model finetune conv layer get accuary 0.779874212337%
